In [ ]:
from bs4 import BeautifulSoup as bs
from bs4 import element
import requests
import csv

### Task #1: get info box for Toy Story 3 film

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# Convert to a beautiful soup object
# soup = bs(r.content)

# Print out the HTML
# contents = soup.prettify()
# print(contents)

### load_info_box function

In [ ]:
def load_info_box(contents):
    soup = bs(r.content)
    # info_rows = soup.find(class_=["infobox vevent", "infobox vcard"])
    info_rows = soup.find(class_="infobox")
    
    if info_rows: 
        info_rows = info_rows.find_all("tr")
    else:
        print ("no class 'infobox vevent' ")
        return -1
    # c = info_box.prettify()
    # info_rows = info_box.
    film_info = {}
    for i,row in enumerate(info_rows):
        if i == 0: 
            film_info['Film Name'] = row.get_text()
        elif i == 1: continue   #skip
        #     # film_info[row.find(class_='infobox-label').get_text()] = row.find(class_= 'infobox-data').get_text()
        # # elif i in range (2,19):  
        else:  
        #     # print (row.prettify())
            try:
                header= row.find("th").get_text() if row.find("th") else ""
                data= row.find("td").get_text() if row.find("td") else ""
            except:
                print ("not found: ", header,  i)
            if header == '': continue
            x = row.find("td")
            if not x:   # no table data
                data = ''
            else:
                x_array = x.find_all('li')  
                if not x_array: # no li, one item
                    data  = x.get_text(" ", strip=True).replace("\xa0", " ") 
                else:   # one item
                    data = [r.get_text(" ", strip=True).replace("\xa0", " ") for r in x_array]
            film_info[header] = data
            # print (i, header, ":", data)
    return film_info

    # print (film_info)
# film_info


## Task #2: Get info box for all movies

### just test

In [ ]:
# tbls = soup.find('h3')
# print (tbls)
# # tbls = soup.select(['h3,'table.wikitable.sortable'])
# tbl = tbls[1]
# print (tbl.i)
# for i in range(1,5):
#         # tbls = tbls.next_element
# #         tbls = tbls.next_sibling
#         print (i,'-',tbls)
# tbls = soup.select('table.wikitable.sortable')
# tbls = soup.select('table.wikitable.sortable')
# tbl = tbls[1]
# print (repr(tbl))
# i = 1
# # tbls = soup.select(['h3','table.wikitable.sortable'])
# tbls = soup.find_all('table', class_= 'wikitable sortable')
# for s in tbls:
#         print (i, '-', repr(s))
#         i +=1
#         # tbls = tbls.next_element
#         # tbl = tbl.prev_sibling
#         # if tbl:
#         #         print (i,'-',tbl['class'])


### testing

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")


# Convert to a beautiful soup object
soup = bs(r.content)

# soup.find_all('td', limit=5) #.find_all('td')
base_url = 'https://en.wikipedia.org'
flog = open('./out/log.txt', 'wt')
movie_info = []
for i,tbl in enumerate(soup.select('table.wikitable.sortable'), start=1):
    # if i == 4: break
    # print (tbl.prev_element)
    # print (tbl.)
    lnks = tbl.select('tbody td i a')
    # print (lnks)
    for j,lnk in enumerate (lnks,start=1):
    #     # if j == 3: break
        url = base_url + lnk['href']

        # r = requests.get(base_url + lnk['href'])
        print (f"table: {i}, row: {j}, {lnk['title']}, {url}")
        flog.write(f"table: {i}, row: {j}: {lnk['title']}, '----->', url\n")
        # a_movie_data = load_info_box(r.content)
        # if a_movie_data == -1:
        #     flog.write (lnk['href'] + "--class 'infobox vevent' not found\n")
        #     continue
        # elif a_movie_data == -2:
        #     flog.write (lnk['href'] + "--class 'iinfobox-data' not found\n")
        #     continue
        # elif a_movie_data == None:
        #     flog.write (lnk['href'] + "**load_info_box returns None \n")
        #     continue
        # movie_info.append(a_movie_data)
        # with open('mycsvfile.csv', 'a') as f:  # append
        #     w = csv.DictWriter(f, a_movie_data.keys())
        #     w.writeheader()
        #     w.writerow(a_movie_data)
flog.close()

### work online/offline for the main page

In [ ]:
# work online
# r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# with open('./out/List_of_Walt_Disney_Pictures_films.html', 'wb') as f:
#     f.write(r.content)

# work offline
with open('./out/List_of_Walt_Disney_Pictures_films.html', 'rb') as f:
    soup = bs(f.read())

# soup.prettify()

### Main routine

In [ ]:
movie_info = []
base_url = 'https://en.wikipedia.org'
flog = open('./out/log.txt', 'wt')
for i,tbl_h in enumerate(soup.select('h3'), start=1):
    # print (tbl_h)
    if i > 10: break    # last table is is 2020s table, rest are not in standard format
    if tbl_h.string:
        # if i > 1: break    #x table for testing, comment when done
        table_header = tbl_h.string
        print (i, 'Table:', table_header)
        # print (tbl_h.nextSibling) # empty
        tbl = tbl_h.nextSibling.nextSibling
        
        if tbl.name != 'table':
            continue
        # print ('tbl type:', type(tbl), tbl.tag, len(tbl) , tbl.keys, tbl.value, tbl.name)
        for j, row in enumerate(tbl.find_all(name='tr'), start=1):
            movie_data = row.select('td:nth-of-type(2)')    # end column
            
            if not movie_data:
                continue

            # print (movie_data)
            try:
                a_tag = movie_data[0].find('a')
                movie_name = a_tag['title']
                movie_link = base_url + a_tag['href']

            except Exception as e:
                # print (f"{i}, {j}: *** Error parsing record: {e}, table: {table_header}, row #: {j}")
                    
                print (f"error: {e} : {table_header}, row: {j}")
                movie_name = movie_data[0].get_text(strip=True)
                movie_link = None

            
            print (i, ',', j, ":", table_header, movie_name, movie_link)
            if not movie_link: 
                flog.write(f"{i},{j}:, {table_header}, {movie_name}, No details \n")
                continue
            r = requests.get(movie_link)
            flog.write(f"{i},{j}:, {table_header}, {movie_name}, {movie_link}\n")
            a_movie_data = load_info_box(r.content)
            if a_movie_data == -1:
                flog.write (movie_link + "--class 'infobox vevent' not found\n")
                continue
            elif a_movie_data == -2:
                flog.write (movie_link + "--class 'infobox-data' not found\n")
                continue
            elif a_movie_data == None:
                flog.write (movie_link + "**load_info_box returns None \n")
                continue

            # print (i, 'Table:', table_header)
            
            full_movie_data = {'table': table_header, 'tbl#':i, "row#": j}
            full_movie_data.update(a_movie_data)
            movie_info.append(full_movie_data)

flog.close()

### testing

In [ ]:
# testing

# r = requests.get("https://en.wikipedia.org/wiki/Zorro_(1957_TV_series)#Theatrical")
# a_movie_data = load_info_box(r.content)
# print (a_movie_data)
len(movie_info)

### Write result DataFrame into csv file for processing

In [ ]:
header = []
for i,row in enumerate(movie_info):
    print 
    for key, val in row.items():
        if key not in header: 
            header.append(key)


with open('./out/mycsvfile-2.csv', 'w', encoding='utf8') as f:  
    w = csv.DictWriter(f, header)
    w.writeheader()
    for i, r in enumerate(movie_info):
        try:
            w.writerow(r)
        except Exception as e:
            print(i, e, r)
            # pass

In [ ]:
import json
def save_data(file_name, data):
    with open(file_name, "w", encoding  = "utf-8") as fp:
        json.dump(movie_info, fp, ensure_ascii = False)

def load_data(file_name, data):
    with open(file_name, "w", encoding  = "utf-8") as fp:
        return json.load(fp)

save_data("./out/movies.json", movie_info)